In [8]:
import requests
import json

HOST = "http://10.35.0.112:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [10]:
cancel=get_cancled()

In [35]:
cancel['data'][0]
map_cabin={ }

{'flight_id': 'c53865e9',
 'departure_airport': 'HBX',
 'arrival_airport': 'IMF',
 'status': 'Cancelled',
 'arrival_time': '2023-12-08T14:00:17.249Z',
 'departure_time': '2023-12-08T12:03:17.249Z'}

In [45]:
get_alt_flights('c53865e9')['data'][0]

{'flight_id': '6246013f',
 'departure_airport': 'IMF',
 'arrival_airport': 'HBX',
 'status': 'Scheduled',
 'arrival_time': '2023-12-08T14:49:17.835Z',
 'departure_time': '2023-12-08T12:10:17.835Z',
 'total_avilable_seats': 52,
 'avilable_seats': {'F': 4,
  'P': 5,
  'C': 3,
  'J': 3,
  'Z': 6,
  'Q': 2,
  'R': 0,
  'S': 0,
  'T': 2,
  'H': 1,
  'M': 3,
  'Y': 1,
  'A': 3,
  'B': 2,
  'D': 0,
  'G': 4,
  'I': 2,
  'E': 1,
  'K': 0,
  'L': 4,
  'N': 1,
  'O': 0,
  'U': 0,
  'V': 1,
  'W': 1,
  'X': 3}}

In [41]:
pnr={}
alt_flights={}
for f in cancel['data']:
#     print(f['flight_id'])
     pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']
     pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']     

In [42]:
pnr

{'c53865e9': [{'pnr': 'e36140', 'score': 2700, 'class': 'F', 'pax': 4},
  {'pnr': '9c112d', 'score': 2550, 'class': 'P', 'pax': 5},
  {'pnr': 'a53d26', 'score': 2350, 'class': 'P', 'pax': 1},
  {'pnr': 'ddb1f4', 'score': 2300, 'class': 'P', 'pax': 4},
  {'pnr': '7d7296', 'score': 2250, 'class': 'F', 'pax': 1},
  {'pnr': 'd7eff6', 'score': 2250, 'class': 'P', 'pax': 1},
  {'pnr': 'c5fa12', 'score': 2250, 'class': 'E', 'pax': 1},
  {'pnr': '99919b', 'score': 2200, 'class': 'F', 'pax': 4},
  {'pnr': '19d7bd', 'score': 2200, 'class': 'F', 'pax': 4},
  {'pnr': '2f2f86', 'score': 2150, 'class': 'P', 'pax': 3},
  {'pnr': '80104a', 'score': 2150, 'class': 'P', 'pax': 1},
  {'pnr': '108834', 'score': 2050, 'class': 'F', 'pax': 1},
  {'pnr': '1e5be2', 'score': 2000, 'class': 'Z', 'pax': 4},
  {'pnr': '010cd0', 'score': 1950, 'class': 'Z', 'pax': 3},
  {'pnr': 'e49fd1', 'score': 1900, 'class': 'C', 'pax': 2},
  {'pnr': 'c253d7', 'score': 1900, 'class': 'J', 'pax': 2},
  {'pnr': '1c1183', 'score':